# OpenCV camera to server to flutter app

- 선재님이 정리한 opencv_sample_codebook 리뷰
    - 스마트폰 카메라 앱을 DAQ로 사용하기 위해 카메라 앱에 접근하는 코드를 정리
**[DAQ]**
- Data Acquisition
- 데이터 수집 시스템을 의미하지만 '데이터수집기/데이터수집장치'라고 생각하면 됨

**[참고]**
- 파이썬 스크립트인 .py가 아인 .ipynb 파일에서는 esc를 눌렀을 때 창이 정상적으로 닫히지 않을 수 있다.

**[조건]**
1. DAQ로 사용할 스마트폰이 코드를 실행하는 컴퓨터(서버)와 같은 네트워크에 연결되어 있어야 함
    - 컴퓨터에 웹캠이 설치되어 있는 경우, 스마트폰과 컴퓨터를 케이블로 연결


In [7]:
# `inet` 이 포함된 ip 주소 모두 확인
# ifconfig | grep -Eo 'inet (addr:)?([0-9]*\.){3}[0-9]*'

In [2]:
import cv2

## 1. 아이폰 카메라 - 컴퓨터 연결
- mac에서 어떤 카메라를 사용할지 index 확인하기

In [5]:
# opencv에서 사용할 카메라 index 확인
for camera_idx in range(-1, 3):
    cap = cv2.VideoCapture(camera_idx)
    if cap.isOpened():
        print(f"Camera found at index: {camera_idx}")
        break

Camera found at index: 0


OpenCV: out device of bound (0-1): -1
OpenCV: camera failed to properly initialize!


- 아이폰을 맥 카메라 리스트에 올려서 사용중인데 여기에서는 카메라리스트에 연동이 안됨
- <img src="iphone_camera_listed_on_mac.PNG" width=30%>

In [3]:
# opencv에서 사용할 카메라 index 확인 - 리스트에 담기
available_cameras = []
for camera_idx in range(-1, 4):
    cap = cv2.VideoCapture(camera_idx)
    if cap.isOpened():
        available_cameras.append(camera_idx)
        print(f"Camera found at index: {camera_idx}")
        cap.release()

print(f"Available camera indexes: {available_cameras}")

2024-03-11 14:42:09.895 python[44020:923518] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
OpenCV: out device of bound (0-1): -1
OpenCV: camera failed to properly initialize!


Camera found at index: 0
Camera found at index: 1
Available camera indexes: [0, 1]


OpenCV: out device of bound (0-1): 2
OpenCV: camera failed to properly initialize!
OpenCV: out device of bound (0-1): 3
OpenCV: camera failed to properly initialize!


- 문제 해결!
    - 리스트에 담아서 출력하지 반복문을 모두 돌고, 두 개의 카메라를 사용할수 있는걸 확인
- *해당 인덱스가 어떤 카메라인지 확인은 어떻게 하지?* 

In [4]:
# import cv2

# Initialize an empty list to store the indexes of available cameras
available_cameras = []

# Iterate over a range of camera indexes (-1 to 3)
for camera_idx in range(-1, 4):
    # Open the video capture device for the current camera index
    cap = cv2.VideoCapture(camera_idx)
    
    # Check if the video capture device is opened successfully
    if cap.isOpened():
        # If opened successfully, add the camera index to the list of available cameras
        available_cameras.append(camera_idx)
        # Print a message indicating that a camera was found at the current index
        print(f"Camera found at index: {camera_idx}")

        # Retrieve camera properties (width, height, frame rate)
        width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
        height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
        fps = cap.get(cv2.CAP_PROP_FPS)

        # Print camera properties
        print(f"Camera {camera_idx} properties:")
        print(f"- Resolution: {int(width)}x{int(height)}")
        print(f"- Frame rate: {int(fps)} FPS")

        # Release the video capture device
        cap.release()

# Print the list of available camera indexes
print(f"Available camera indexes: {available_cameras}")


OpenCV: out device of bound (0-1): -1
OpenCV: camera failed to properly initialize!


Camera found at index: 0
Camera 0 properties:
- Resolution: 1280x720
- Frame rate: 30 FPS
Camera found at index: 1
Camera 1 properties:
- Resolution: 1920x1080
- Frame rate: 30 FPS
Available camera indexes: [0, 1]


OpenCV: out device of bound (0-1): 2
OpenCV: camera failed to properly initialize!
OpenCV: out device of bound (0-1): 3
OpenCV: camera failed to properly initialize!


In [ ]:
# 웹캠 영상을 캡처하고 화면에 표시하는 함수 정의
def show_video():
    # VideoCapture 객체 생성 - 핸드폰 카메라 사용
    cap = cv2.VideoCapture(1)

    # 프레임 가로 크기 지정
    cap.set(3, 960)
    # 프레임 세로 크기 지정
    cap.set(4, 640)

    # 카메라로부터 프레임을 읽어오기
    ret, frame = cap.read()
    
    if not ret:
        # 프레임 읽기 실패
        print("Error: Failed to capture frame")
    else:
        # 종료될 때까지 계속해서 프레임 읽기
        while True:
            ret, frame = cap.read()
            # video라는 이름의 창에 프레임 출력
            cv2.imshow('video', frame)

            # 입력되는 키를 k변수에 저장 - 1초마다 사용자 입력을 기다림
            k = cv2.waitKey(1) & 0xff1
            if k == 27: # 27은 esc키 번호
                # esc 키가 눌리면 프레임 읽기 종료
                break
        
        # VideoCapture 객체 소멸
        cap.release()
        # 창 닫기
        cv2.destroyAllWindows()

# show_video 함수 호출
show_video()

2024-03-11 14:45:38.356 python[44076:927130] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


## 2. 카메라로 촬영한 영상 저장하기
OpenCV 라이브러리를 사용하여 스마트폰 카메라로 촬영되는 영상을 컴퓨터(서버)에 저장

In [ ]:

import cv2  # openCV 라이브러리
from datetime import datetime  # 파일명에 시간을 추가하기 위한 datetime 라이브러리
import os  # 폴더 생성을 위해 추가


# 웹캠 영상을 캡처하고 영상 데이터로 저장하는 함수 정의
def write_video():
    # VideoCapture 객체 생성 - 0: 맥북 카메라, 1: 핸드폰 카메라
    cap = cv2.VideoCapture(1)

    # 프레임 가로 크기 지정
    cap.set(3, 960)
    # 프레임 세로 크기 지정
    cap.set(4, 480)

    # 초당 프레임 수 설정
    fps = 20
    
    # 저장되는 영상의 가로 크기를 프레임의 가로 크기로 지정
    width = int(cap.get(3))
    # 저장되는 영상의 세로 크기를 프레임의 세로 크기로 지정
    height = int(cap.get(4))
    # macOS MP4 코덱인 MP4V로 FourCC 코드 설정
    fcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')

    # 현재 시간을 포맷에 맞게 문자열로 변환하여 파일명에 추가
    current_time = datetime.now().strftime("%d_%H%M%S")
    # 파일명에 현재 시간 추가
    file_name = f"video_test_{current_time}.mp4"

    folder_path = "./video"

    # 만약 폴더가 존재하지 않으면 폴더 생성
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # VideoWriter 객체 생성 - 영상 저장 경로 설정
    out = cv2.VideoWriter(os.path.join(folder_path, file_name), fcc, fps, (width, height))

    # 카메라로부터 프레임 읽어오기
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame")
    else:
        while True:
            ret, frame = cap.read()

            # video_test라는 이름의 창에 프레임 출력
            cv2.imshow('video_test', frame)
            
            # 파일 저장
            out.write(frame)  
            
            # 입력되는 키를 k변수에 저장 - 1초마다 사용자 입력을 기다림
            k = cv2.waitKey(int(1000/fps)) & 0xff # 27은 esc키 번호
            if k == 27:
                # esc 키가 눌리면 프레임 읽기 종료
                break

            
        # VideoCapture 객체 소멸
        cap.release()
        # VideoWriter 객체 소멸
        out.release()
        # 창 닫기
        cv2.destroyAllWindows()


# write_video 함수 호출            
write_video()